Notebook showing usage of the data detective.

In [1]:
# To import detective from relative folder
import detective.core as detective
import detective.functions as functions
import pandas as pd
import os

To load the local db we supply the absolute path to the sqlite `.db` file. Make sure the path is correct, as if the path is incorrect an empty db file we be created at the path you specified and you will experience errors as it contains. Note that if using docker the path will be something like the one here. 

In [2]:
cwd = os.getcwd()
cwd

'/workspaces/HASS-data-detective'

In [3]:
db_path = f"{cwd}/notebooks/home-assistant_v2.db"
db_mb = os.path.getsize(db_path)/1e6
print(f"The size of {db_path} is {db_mb} MB")

The size of /workspaces/HASS-data-detective/notebooks/home-assistant_v2.db is 2.56 MB


In [4]:

db = detective.HassDatabase('sqlite:///{db_path}') # To init without fetching entities fetch_entities=False

Successfully connected to database sqlite:///{db_path}
Error with query: 
            SELECT DISTINCT(entity_id) FROM states
            
(sqlite3.OperationalError) no such table: states
[SQL: 
            SELECT DISTINCT(entity_id) FROM states
            ]
(Background on this error at: http://sqlalche.me/e/e3q8)


OperationalError: (sqlite3.OperationalError) no such table: states
[SQL: 
            SELECT DISTINCT(entity_id) FROM states
            ]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [0]:
db.entities

Use `fetch_all_sensor_data()` to fetch all your sensor data into a pandas dataframe in memory. Note that by default the number of states returned is limited but this can optionally be over-ridden as shown below. 

In [0]:
df = db.fetch_all_sensor_data(limit=1e9)

Take a minute to read about the differences between [wide-form and long-form data](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data). The Pandas dataframe we have is in long form.

In [0]:
df.head()

It is necessary to do some formatting of the data before we can plot it, and detective provides several functions to assist. Additionally it will generate some features for you.

In [0]:
df = functions.generate_features(df)
df = functions.format_dataframe(df)

In [0]:
df.head()

Notice the new feature columns added. It is straightforward to create your own features, for example to add a `day_of_week` column

In [0]:
df['day_of_week'] = df['last_changed'].apply(lambda x : x.dayofweek)

In [0]:
df.head()

## Plot some data
First plot using [Seaborn](https://seaborn.pydata.org/)

In [0]:
#!pip install seaborn # Uncomment to install if required

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

to_plot = 'particles_01um'
fig, ax = plt.subplots(1, figsize=(10,6))
sns.lineplot(
    x='last_changed', 
    y='state', 
    hue='entity_id', 
    data=df[df['friendly_name'] == to_plot], 
    ax=ax);

Now using [Altair](https://altair-viz.github.io/index.html)

In [0]:
#!pip install altair # Uncomment to install altair

In [0]:
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

to_plot = 'particles_01um'
alt.Chart(df[df['friendly_name'] == to_plot]).mark_line().encode(
    x='last_changed',
    y='state',
    color='entity_id',
    tooltip=['entity_id', 'state', 'last_changed']
).properties(
    width=800,
    height=300
).interactive()